In [1]:
import pandas as pd
import subprocess
import json
import xmltodict
import pprint
import numpy as np
# import  aspose.cells 
# from aspose.cells import Workbook

pd.set_option('max_colwidth',250)
pd.set_option('display.max_rows',90)

todosCargos = Contem os cargos de cada comissão

In [15]:
def cargos(todos,novos,codComissao):
    lista = pd.DataFrame()
    for i,dic in enumerate(novos):
        aux = pd.DataFrame.from_dict(dic,orient='index').T
        aux["codComissao"] = codComissao
        lista = pd.concat([aux,lista])
    lista = pd.concat([todos,lista])
    return lista

In [65]:
comando = 'curl https://legis.senado.leg.br/dadosabertos/dados/ConselhosOrgaosCongresso.xml'
resultado = subprocess.run(comando, shell=True, text=True, capture_output=True)
resultado = resultado.stdout
resultado = json.dumps(xmltodict.parse(resultado))
resultado = pd.read_json(resultado)
resultado = resultado.loc["Colegiados"]["ConselhosOrgaosCongresso"]
resultado = pd.DataFrame(resultado)
lista = pd.DataFrame()
todosCargos = pd.DataFrame()
for i,r in resultado.iterrows():
    aux = pd.DataFrame(r.loc["Colegiado"])
    lista = pd.concat([lista, aux], axis=0)
lista = lista.groupby("CodigoColegiado").first()
#dados = pd.read_xml()

C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_21208\3466073604.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  resultado = pd.read_json(resultado)


In [67]:
listafor 

,NomeColegiado,SiglaColegiado,Finalidade,DataInicio,QuantidadesMembros,Cargos,Eventos,MembrosBlocoSF,MembrosBlocoCD,EventosProrrogacao,ObservacoesVaga,InformacoesSecretaria,TipoColegiado,DescricaoSubtitulo
CodigoColegiado,,,,,,,,,,,,,,
1916,Representação Brasileira no Parlamento do Mercosul,CPCMS,A Representação Brasileira no Parlamento do Mercosul tem por finalidade ser o órgão de ligação entre o Congresso Nacional e o Parlamento do Mercosul.,02/02/2015,37,"[{'TipoCargo': 'PRESIDENTE', 'CodigoCargo': '1', 'NomeParlamentar': 'Senador Nelsinho Trad', 'CodigoParlamentar': '5985', 'Bancada': '(PSD-MS)'}, {'TipoCargo': '1º VICE-PRESIDENTE', 'CodigoCargo': '4', 'NomeParlamentar': 'Senador Humberto Costa',...","[{'TipoEvento': 'Designação', 'DataEvento': '11/07/2023', 'OrdemImpressao': '1'}, {'TipoEvento': 'Instalação', 'DataEvento': '13/09/2023', 'OrdemImpressao': '2'}]","[{'UnidadePartidaria': 'Bloco Parlamentar Democracia', 'NumeroMembros': '8', 'PartidosBloco': {'Partidos': [{'SiglaPartido': 'UNIÃO'}, {'SiglaPartido': 'MDB'}, {'SiglaPartido': 'PODEMOS'}, {'SiglaPartido': 'PDT'}, {'SiglaPartido': 'PSDB'}]}, 'Mem...","[{'UnidadePartidaria': 'PL', 'NumeroMembros': '10', 'MembrosCD': {'Membro': [{'ProprietarioVaga': 'PL', 'NomeParlamentar': 'Deputado Giovani Cherini', 'CodigoParlamentar': '5336', 'CodigoDeputadoNaCamara': '160673', 'SiglaUf': 'RS', 'Partido': 'P...","{'Leitura': '02/02/2015', 'TipoEvento': {'Membro': [{'TipoEvento': 'Designação', 'DatEvento': '11/07/2023', 'CodigoColegiado': '1916'}, {'TipoEvento': 'Instalação', 'DatEvento': '13/09/2023', 'CodigoColegiado': '1916'}]}}","[{'Linha': '1', 'ObservacaoParlamentar': '11/07/2023: Designado como suplente o Senador Wellington Fagundes, conforme ATCN 49/2023 e Ofício n° 125/2023 do Bloco Vanguarda.'}, {'Linha': '2', 'ObservacaoParlamentar': '11/07/2023: Designado como tit...",Antônio Ferreira Costa Filho,Órgão,None
767,Conselho de Comunicação Social,CCS,"O Conselho de Comunicação Social tem como atribuição a realização de estudos, pareceres, recomendações e outras solicitações que lhe forem encaminhadas pelo Congresso Nacional a respeito do Título VIII, Capítulo V, da Constituição Federal (art. 2...",30/12/1991,None,"[{'TipoCargo': 'PRESIDENTE', 'CodigoCargo': '1', 'NomeParlamentar': 'Miguel Matos', 'Bancada': '(-)'}, {'TipoCargo': 'VICE-PRESIDENTE', 'CodigoCargo': '2', 'NomeParlamentar': 'Patricia Blanco', 'Bancada': '(-)'}]","[{'TipoEvento': 'Eleição Geral', 'DataEvento': '05/06/2002', 'OrdemImpressao': '1'}, {'TipoEvento': 'Eleição Geral', 'DataEvento': '22/12/2004', 'OrdemImpressao': '1'}, {'TipoEvento': 'Eleição Geral', 'DataEvento': '17/07/2012', 'OrdemImpressao':...","[{'NumeroMembros': '2', 'MembrosSF': {'Membro': [{'ProprietarioVaga': 'Representante das empresas de rádio (inciso I)', 'NomeParlamentar': 'Flavio Lara Resende', 'TipoVaga': 'Titular', 'NumeroOrdem': '1', 'OrigemParlamentar': 'Congresso'}, {'Prop...",None,{'Leitura': '30/12/1991'},None,SECRETARIA-GERAL DA MESA,Conselho,"Lei nº 8.389, de 1991,\n Regimento Interno aprovado nos termos do Ato da Mesa nº 1, de 2013"
768,Conselho da Ordem do Congresso Nacional,COCN,"A Ordem do Congresso Nacional é destinada a prestigiar as pessoas físicas ou jurídicas, nacionais ou estrangeiras, que se tenham tornado dignas do especial reconhecimento do Poder Legislativo do Brasil. Compete ao Conselho aprovar ou rejeitar as ...",23/11/1972,22,None,"[{'TipoEvento': 'Atualização', 'DataEvento': '12/09/2012', 'OrdemImpressao': '8'}, {'TipoEvento': 'Atualização', 'DataEvento': '04/02/2013', 'OrdemImpressao': '8'}, {'TipoEvento': 'Atualização', 'DataEvento': '07/03/2013', 'OrdemImpressao': '8'},...",None,None,{'Leitura': '23/11/1972'},None,"Núcleo de Premiações, Frentes e Grupos Parlamentares - NPFG",Conselho,"Decreto Legislativo nº 70, de 1972, regulamentado pelo Ato nº 1, de 1973-CN"
769,Conselho do Diploma do Mérito Educativo Darcy Ribeiro,DMEDR,O Diploma do Mérito Educativo Darcy Ribeiro é entregue pelo Congresso Nacional a pess

In [31]:
todosCargos = pd.DataFrame()
for i,r in lista.iterrows():
    try:
        todosCargos = cargos(todosCargos,r["Cargos"],r["CodigoColegiado"])
    except Exception as e:
        del e

In [48]:
def membros(todos,novos,codComissao):
    lista = pd.DataFrame()
    for i,dic in enumerate(novos):
        # print(dic)
        aux = pd.DataFrame.from_dict(dic,orient='index').T
        lista = pd.concat([aux,lista])
    lista = pd.concat([todos,lista])
    return lista